In [11]:
import pandas as pd
import os

In [12]:
# 1. Define the root directory containing yearly folders
root_dir = "/Users/jasminehuang/Desktop/datasets to be used/alerts 2020-2024"  # Replace with the path to your 'alerts 2020-2024' folder

# 2. Traverse all subfolders and collect CSV file paths
csv_files = []
for subdir, _, files in os.walk(root_dir):  # os.walk will traverse subfolders recursively
    for file in files:
        if file.endswith('.csv'):
            csv_files.append(os.path.join(subdir, file))

# 3. Combine all CSV files into a single DataFrame
combined_df = pd.concat(
    [pd.read_csv(file, dtype=str, low_memory=False) for file in csv_files],
    ignore_index=True
)

In [13]:
# Clean route_type column: remove whitespaces and drop NaN
combined_df['route_type'] = combined_df['route_type'].str.strip()
combined_df = combined_df.dropna(subset=['route_type'])

# Filter for route_type '1' and '0'
subway_df = combined_df[(combined_df['route_type'] == '1') | (combined_df['route_type'] == '0')]

In [14]:
print("Unique route_type values:")
print(subway_df['route_type'].unique())

Unique route_type values:
['1' '0']


In [15]:
alert_summary = subway_df.groupby(['route_id', 'cause']).size().reset_index(name='count')

# Find the most common alert per line
most_common_alerts = alert_summary.loc[alert_summary.groupby('route_id')['count'].idxmax()]

# Optional: Preview the results
print(most_common_alerts)

    route_id              cause  count
1       Blue       CONSTRUCTION   6021
10   Green-B       CONSTRUCTION  79910
23   Green-C  TECHNICAL_PROBLEM  12811
27   Green-D        MAINTENANCE  87214
40   Green-E  TECHNICAL_PROBLEM  13225
48  Mattapan  TECHNICAL_PROBLEM    436
53    Orange        MAINTENANCE  13161
61       Red        MAINTENANCE  45396


In [10]:
data_as_dict = most_common_alerts.set_index("route_id").to_dict(orient="index")

# Write the dictionary to JSON
import json
with open("alerts.json", "w") as f:
    json.dump(data_as_dict, f, indent=4)